In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import torch
import test_fn

In [11]:
import importlib
importlib.reload(test_fn)

<module 'test_fn' from 'C:\\Users\\Andrew\\OneDrive\\Documents\\HesBO\\baseline\\test_fn.py'>

# Draw sample inputs

In [12]:
n = 100
low_dim = 50
fn_dim = 200
fn = test_fn.Ackley()

In [13]:
x = torch.rand((n,low_dim))
A = torch.rand((low_dim, fn_dim))
X = x@A
y = fn.evaluate(X)

In [14]:
X[torch.topk(y, 3, largest = False).indices,:].shape

torch.Size([3, 200])

In [15]:
X.shape

torch.Size([100, 200])

In [16]:
fn = test_fn.Levy(10)

# Algorithm 1: Shift subspace to best performing points

In [18]:
importlib.reload(test_fn)
fn = test_fn.Ackley()
x = torch.rand((100,low_dim))
A = torch.rand((low_dim, fn_dim))
X = x@A
y = fn.evaluate(X)

# Find best performing points (Alt: Find best performing and nearest neighbors)
best_performers_indices = torch.topk(y, 1, largest = False).indices
best_performers = X[best_performers_indices]

# Update
A = best_performers

In [70]:
def learn_subspace_best_performers(fn, low_dim, fn_dim, 
                                   num_best = 1, num_neighbors = 0,
                                   move_subspace = True,
                                   step_size = 1,
                                   restarts = 0,
                                   samples = 200, max_iter = 1000, trace = False):
    
    best_found = 1e10
    A = torch.randn((low_dim, fn_dim))
    B = torch.randn(fn_dim)
    first_run = True
    
    for i in range(max_iter):
        # Evaluate
        x = (torch.rand((samples,low_dim)) * 2) - 1
        X = x@A + B
        y = fn.evaluate(X)

        # Find best performing points (Alt: Find best performing and nearest neighbors)
        best_performers_indices = torch.topk(y, num_best, largest = False).indices
        
        if num_neighbors > 0:
            for j in best_performers_indices:
                distances = torch.sum((X - X[j,:])**2, dim = 1)
                neighbor_indices = torch.topk(distances, num_neighbors, largest = False).indices
                neighbor_indices = neighbor_indices[1:]
                best_performers_indices = torch.concat([best_performers_indices, neighbor_indices])
        
        shift_point = X[best_performers_indices] * step_size
        
        

        # Update
        if move_subspace:
            A = torch.concat([shift_point, torch.randn((low_dim - num_best, fn_dim))], dim = 0)
        else:
            if trace:
                print(shift_point.shape)
            B = torch.mean(shift_point, dim = 0)
            A = torch.randn((low_dim, fn_dim))

        # Iterate best found (searching for minimum)
        if min(y) < best_found:
            best_point = X[best_performers_indices[0]]
            best_found = min(y)

        if i%int(max_iter/10) == 0:
            print(f"Current min/best min found y is:{min(y):.2f}, {best_found:.2f}")
            
#     print(f"Best point found at {best_point}")

In [71]:
learn_subspace_best_performers(fn, 50, 200, 
                               num_best = 1, num_neighbors = 0,
                               move_subspace = True,
                               step_size = 1,
                               restarts = 0)

Current min/best min found y is:11.68, 11.68
Current min/best min found y is:11.71, 10.98
Current min/best min found y is:11.38, 10.81
Current min/best min found y is:11.91, 10.81
Current min/best min found y is:11.72, 10.67
Current min/best min found y is:11.42, 10.67
Current min/best min found y is:11.69, 10.67
Current min/best min found y is:11.04, 10.67
Current min/best min found y is:12.06, 10.24
Current min/best min found y is:11.55, 10.24


In [72]:
learn_subspace_best_performers(fn, 50, 200, 
                               num_best = 2, num_neighbors = 0,
                               move_subspace = False,
                               step_size = 1,
                               restarts = 0)

Current min/best min found y is:11.29, 11.29
Current min/best min found y is:16.90, 11.29
Current min/best min found y is:16.29, 11.29
Current min/best min found y is:16.66, 11.29
Current min/best min found y is:16.73, 11.29
Current min/best min found y is:16.75, 11.29
Current min/best min found y is:16.97, 11.29
Current min/best min found y is:16.64, 11.29
Current min/best min found y is:16.49, 11.29
Current min/best min found y is:16.20, 11.29


In [73]:
learn_subspace_best_performers(fn, 50, 200, 
                               num_best = 2, num_neighbors = 0,
                               move_subspace = False,
                               step_size = 0.5,
                               restarts = 0)

Current min/best min found y is:11.47, 11.47
Current min/best min found y is:11.41, 10.67
Current min/best min found y is:11.58, 10.55
Current min/best min found y is:11.18, 10.36
Current min/best min found y is:11.51, 10.36
Current min/best min found y is:11.94, 10.36
Current min/best min found y is:12.10, 10.36
Current min/best min found y is:11.69, 10.18
Current min/best min found y is:11.75, 10.18
Current min/best min found y is:11.90, 10.18


In [74]:
learn_subspace_best_performers(fn, 50, 200, 
                               num_best = 2, num_neighbors = 1,
                               move_subspace = True,
                               step_size = 0.8,
                               max_iter = 10000,
                               restarts = 0)

Current min/best min found y is:11.35, 11.35
Current min/best min found y is:12.05, 10.45
Current min/best min found y is:11.54, 10.45
Current min/best min found y is:11.65, 10.45
Current min/best min found y is:11.38, 10.45
Current min/best min found y is:11.22, 10.40
Current min/best min found y is:11.69, 10.01
Current min/best min found y is:11.57, 10.01
Current min/best min found y is:11.74, 10.01
Current min/best min found y is:11.99, 9.96


In [75]:
learn_subspace_best_performers(fn, 4, 200, 
                               num_best = 2, num_neighbors = 0,
                               move_subspace = True,
                               step_size = 0.8,
                               max_iter = 10000,
                               restarts = 0)

Current min/best min found y is:5.26, 5.26
Current min/best min found y is:1.54, 1.54
Current min/best min found y is:4.68, 1.54
Current min/best min found y is:3.48, 1.35
Current min/best min found y is:3.53, 1.35
Current min/best min found y is:3.91, 1.35
Current min/best min found y is:3.95, 1.35
Current min/best min found y is:3.56, 1.24
Current min/best min found y is:2.39, 1.24
Current min/best min found y is:4.90, 1.24


In [76]:
learn_subspace_best_performers(fn, 4, 200, 
                               num_best = 2, num_neighbors = 0,
                               move_subspace = False,
                               step_size = 0.8,
                               max_iter = 10000,
                               restarts = 0)

Current min/best min found y is:5.19, 5.19
Current min/best min found y is:3.29, 2.33
Current min/best min found y is:2.81, 2.33
Current min/best min found y is:3.66, 2.24
Current min/best min found y is:2.87, 2.24
Current min/best min found y is:3.12, 2.24
Current min/best min found y is:3.34, 2.24
Current min/best min found y is:3.62, 2.24
Current min/best min found y is:3.26, 2.24
Current min/best min found y is:3.38, 2.24


In [77]:
learn_subspace_best_performers(fn, 4, 200, 
                               num_best = 2, num_neighbors = 0,
                               move_subspace = True,
                               step_size = 0.8,
                               max_iter = 10,
                               restarts = 0)

Current min/best min found y is:5.35, 5.35
Current min/best min found y is:1.54, 1.54
Current min/best min found y is:4.92, 1.54
Current min/best min found y is:3.02, 1.54
Current min/best min found y is:5.04, 1.54
Current min/best min found y is:3.38, 1.54
Current min/best min found y is:4.96, 1.54
Current min/best min found y is:2.83, 1.54
Current min/best min found y is:4.70, 1.54
Current min/best min found y is:3.10, 1.54
